# Phân tích và Dự đoán Sống sót trên tàu Titanic
**Thư viện cần thiết:**

- `pandas`
- `numpy`
- `matplotlib`, `seaborn`
- `sklearn`

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Xem thử dữ liệu

In [2]:
try:
    train_df = pd.read_csv('../../data/train_imputed_method2.csv')
    test_df = pd.read_csv('../../data/test_imputed_method2.csv')
except FileNotFoundError:
    print("LỖI: Không tìm thấy file dữ liệu. Hãy đảm bảo bạn có cấu trúc thư mục 'data/...' chính xác.")
    raise

print("Dữ liệu train:")
display(train_df.head())

print("\nDữ liệu test:")
display(test_df.head())

Dữ liệu train:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,HasCabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,0



Dữ liệu test:


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,HasCabin
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs,0


# Chuẩn bị dữ liệu cho mô hình

1.  Tách biến mục tiêu `Survived` (cái chúng ta muốn dự đoán) ra khỏi tập huấn luyện.
2.  Xác định các biến đặc trưng (features) sẽ được dùng làm đầu vào cho mô hình.
3.  Lưu lại `PassengerId` của tập test để dùng cho file kết quả cuối cùng.

In [3]:
# 1. Xác định các đặc trưng bạn muốn sử dụng
features = ['Pclass', 'Sex', 'Age', 'Fare']

# 2. Xác định biến mục tiêu (y)
y_train = train_df['Survived']

# 3. Tạo bản sao của các đặc trưng đã chọn để tránh thay đổi dữ liệu gốc
X_train = train_df[features].copy()
X_test = test_df[features].copy()

# 4. Chuyển đổi cột 'Sex' thành dạng số (0 và 1)
# Dùng map: 'male' -> 0, 'female' -> 1
sex_mapping = {'male': 0, 'female': 1}
X_train['Sex'] = X_train['Sex'].map(sex_mapping)
X_test['Sex'] = X_test['Sex'].map(sex_mapping)

# 5. Lưu lại PassengerId để tạo file submission
test_passenger_ids = test_df['PassengerId']

# In ra để kiểm tra
print("Các cột được sử dụng làm đặc trưng (features):")
print(X_train.columns)
print("\nKiểu dữ liệu của các cột sau khi chuyển đổi 'Sex':")
X_train.info()

print(f"\nKích thước của X_train: {X_train.shape}")
print(f"Kích thước của y_train: {y_train.shape}")
print(f"Kích thước của X_test: {X_test.shape}")

Các cột được sử dụng làm đặc trưng (features):
Index(['Pclass', 'Sex', 'Age', 'Fare'], dtype='object')

Kiểu dữ liệu của các cột sau khi chuyển đổi 'Sex':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    int64  
 2   Age     891 non-null    float64
 3   Fare    891 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 28.0 KB

Kích thước của X_train: (891, 4)
Kích thước của y_train: (891,)
Kích thước của X_test: (418, 4)


# Tách tập dữ liệu huấn luyện thành tập huấn luyện và tập kiểm định (Validation)

- `X_train_split` (80%): Dùng để huấn luyện mô hình.
- `X_val` (20%): Dùng để kiểm tra hiệu suất của mô hình trên dữ liệu mà nó chưa từng thấy.

In [4]:
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)
# stratify=y_train giúp đảm bảo tỷ lệ Sống sót/Không sống sót trong tập train và val là tương đương nhau.

print("Kích thước tập huấn luyện mới:", X_train_split.shape)
print("Kích thước tập kiểm định (validation):", X_val.shape)

Kích thước tập huấn luyện mới: (712, 4)
Kích thước tập kiểm định (validation): (179, 4)


# Xây dựng và Huấn luyện mô hình KNN

Bây giờ chúng ta sẽ khởi tạo và huấn luyện mô hình trên tập dữ liệu `X_train_split`.

In [5]:
# Bước 1: Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_split_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val) # Chỉ .transform() trên tập validation

# Bước 2: Khởi tạo mô hình KNN
# n_neighbors=5 là số láng giềng gần nhất được dùng để bỏ phiếu
knn_model = KNeighborsClassifier(n_neighbors=5)

# Bước 3: Huấn luyện mô hình trên dữ liệu đã được chuẩn hóa
knn_model.fit(X_train_split_scaled, y_train_split)

print("Mô hình KNN đã được huấn luyện thành công!")

Mô hình KNN đã được huấn luyện thành công!


# Đánh giá hiệu suất của mô hình

Chúng ta sẽ sử dụng tập kiểm định (`X_val`, `y_val`) để xem mô hình dự đoán chính xác đến mức nào.

In [10]:
# Dự đoán trên tập kiểm định đã được chuẩn hóa
y_pred_val = knn_model.predict(X_val_scaled) # <--- Sửa ở đây

# Tính toán và in ra độ chính xác
accuracy = accuracy_score(y_val, y_pred_val)
print(f"Độ chính xác trên tập kiểm định: {accuracy * 100:.2f}%")

# In ra báo cáo chi tiết hơn (precision, recall, f1-score)
print("\nBáo cáo phân loại chi tiết:")
print(classification_report(y_val, y_pred_val))

Độ chính xác trên tập kiểm định: 81.56%

Báo cáo phân loại chi tiết:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       110
           1       0.78      0.72      0.75        69

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.82      0.81       179



# Tạo file dự đoán cuối cùng

Mô hình đã được đánh giá. Bây giờ, chúng ta sẽ huấn luyện lại nó trên **toàn bộ tập dữ liệu huấn luyện** để mô hình học được nhiều thông tin nhất có thể, sau đó dùng nó để dự đoán trên tập `X_test`.

In [11]:
# Bước 1: Chuẩn hóa toàn bộ dữ liệu train và test
final_scaler = StandardScaler()
X_train_scaled = final_scaler.fit_transform(X_train)
X_test_scaled = final_scaler.transform(X_test)

# Bước 2: Huấn luyện lại mô hình KNN trên toàn bộ dữ liệu train đã chuẩn hóa
print("Huấn luyện mô hình KNN cuối cùng trên toàn bộ dữ liệu train...")
final_model = KNeighborsClassifier(n_neighbors=5)
final_model.fit(X_train_scaled, y_train)
print("-> Huấn luyện hoàn tất.")

# Bước 3: Dự đoán kết quả cho tập test đã chuẩn hóa
print("Dự đoán trên dữ liệu test...")
test_predictions = final_model.predict(X_test_scaled)
print("-> Dự đoán hoàn tất.")

# Bước 4: Tạo DataFrame submission
submission = pd.DataFrame({
    "PassengerId": test_passenger_ids,
    "Survived": test_predictions
})

# Bước 5: Lưu file kết quả vào thư mục mới
submission.to_csv('../../models/knn/submission.csv', index=False)

print("File được lưu tại: models/knn/submission.csv")
display(submission.head())

Huấn luyện mô hình KNN cuối cùng trên toàn bộ dữ liệu train...
-> Huấn luyện hoàn tất.
Dự đoán trên dữ liệu test...
-> Dự đoán hoàn tất.
File được lưu tại: models/knn/submission.csv


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
